# Table of Contents
* [1 Importing Libraries](#1)
* [2 Prepare Data](#2)
    * [2.1 Generating passengers with poisson distribution](#2.1)
* [3 Define Process](#3)
    * [3.1 Blocks](#3.1)
    * [3.2 Utilities](#3.2)
* [4 Simulation](#4)
* [5 Statistics](#5)
* [6 Optimization](#6)
    * [6.1 Monte-Carlo optimisation to minimise the overall average travelling time](#6.1)
    * [6.2 Monte-Carlo optimisation to maximize passengers throughput](#6.2)
* [7 References](7)

# Importing Libraries
<a id='1'></a>

In [129]:
import pandas as pd
import numpy as np
import math
import random

from matplotlib import pyplot as plt

import simpy
import simpy.events as evt

# Prepare Data
<a id='2'></a>

In [130]:
df = pd.DataFrame({'From': ['London Euston', 'London Old Oak Common', 'Birmingham Interchange'],
                   'To' : ['London Old Oak Common', 'Birmingham Interchange', 'Birmingham Curzon Street'],
                   'Traveling time' : [5, 31, 9],
                   'Distance' : [30, 145, 35],
                   'Blocks': [1, 14, 2],
                  'Dwell time' : [2, 2, 2]})
df

,From,To,Traveling time,Distance,Blocks,Dwell time
0,London Euston,London Old Oak Common,5,30,1,2
1,London Old Oak Common,Birmingham Interchange,31,145,14,2
2,Birmingham Interchange,Birmingham Curzon Street,9,35,2,2


Convert distance to meters and traveling time to seconds

In [131]:
df['Distance']=df['Distance'].apply(lambda x: int(x*1000))
df['Traveling time']=df['Traveling time'].apply(lambda x: int(x*60))

In [132]:
df

,From,To,Traveling time,Distance,Blocks,Dwell time
0,London Euston,London Old Oak Common,300,30000,1,2
1,London Old Oak Common,Birmingham Interchange,1860,145000,14,2
2,Birmingham Interchange,Birmingham Curzon Street,540,35000,2,2


## Generating passengers with poission distribution
<a id='2.1'></a>

In [133]:
def poissonEvents(λ=None, N=None, T=None, 
                  plot=True, events=None, figsize=None):

    if events!=None:
        N = len(events)
        
    if N!=None and T==None and λ!=None:
        T = int(N/λ)
    elif N==None and T!=None and λ!=None:
        N = int(λ*T)
    elif N!=None and T!=None and λ==None:
        λ = N/T
    
   
    if events==None:
        u = [ random.uniform(0, 1) for i in range(N) ]
        P = list(np.cumsum(list(map(
                lambda x: -math.log(1-x)/λ, u))))
    else:
        P = events
        
    if plot:
  
        if figsize!=None:
            width, height = figsize
        else:
            width, height = 10, 6
            
        fig, ax=plt.subplots(1,1)
        fig.set_figwidth(width)
        fig.set_figheight(height)
        
        def X(l):
            
            xmax = max(int(N/λ),math.ceil(max(l)))
            
            def double(l):
                return [] if l==[] \
                        else [l[0], l[0]]+double(l[1:])
    
            return [0]+double(l)+[xmax]
        
        def Y(l):
    
            def steps(l, n):
                return [] if l==[] \
                        else [n, n]+steps(l[1:], n+1)
    
            return [0, 0]+steps(l, 1)
    
        x = X(P)
        y = Y(P)

        ax.set_title(f"Poisson Process λ={λ:5.3f} n={N:d}")
        ax.set_xlim(min(x), max(x))
        ax.set_ylim(min(y), max(y))
        ax.yaxis.set_major_locator(
            mpl.ticker.MaxNLocator(integer=True))
        ax.plot(x, y, lw=3)
        ax.plot(x, list(map(lambda x:λ*x, x)))
        ax.grid(True)
        
    return P

Generating passengers arriving for each station. We assume 300 passengers per hour.

For every iteration call a new set of passengers.

In [134]:
def passenger_generator():
    global passenger_arrival
    s1data = pd.DataFrame()
    s1data['station'] = np.repeat('London Euston', 1000)
    s1data['arr_time'] = poissonEvents(λ=300/3600, N=1000, plot=False)

    s2data = pd.DataFrame()
    s2data['station'] = np.repeat('London Old Oak Common', 1000)
    s2data['arr_time'] = poissonEvents(λ=300/3600, N=1000, plot=False)

    s3data = pd.DataFrame()
    s3data['station'] = np.repeat('Birmingham Interchange', 1000)
    s3data['arr_time'] = poissonEvents(λ=300/3600, N=1000, plot=False)

    passenger_arrival = pd.concat([s1data, s2data, s3data])

    a = [round(x,2) for x in passenger_arrival['arr_time']]
    passenger_arrival['arr_times'] = a
    passenger_arrival = passenger_arrival.drop(columns=['arr_time'])
    passenger_arrival.head()

    return passenger_arrival

In [135]:
passenger_generator()

,station,arr_times
0,London Euston,19.19
1,London Euston,20.23
2,London Euston,20.27
3,London Euston,21.82
4,London Euston,27.69
...,...,...
995,Birmingham Interchange,11701.43
996,Birmingham Interchange,11710.51
997,Birmingham Interchange,11713.84
998,Birmingham Interchange,11731.02


Dataframe for train schedule to be used later to evaluate results for optimisation

In [136]:
London_Birmingham = pd.DataFrame(columns=['Train', 'Departure_time', 'Arrival_time', 'Passengers'])

# Define Process
<a id='3'></a>

The train class models the train process and the entire train journey from London to Birmingham station is in the process method. In this simualtion, we assume that the trains are travelling at the same speed.

To start the journey, the train takes passengers on board, we're giving 60 seconds for this to happen. After boarding passengers, the train requests the signaling block to leave for the next station. As block is a Simpy resource, if the block is in use by another train, the train will wait until the block is no longer in use. This process is iterative until the train arrives at the final destination Birmingham station. 

We record the train schedule: train number, departure time, arrival time and number of passengers during the process.

In [137]:
class Train(object):  
    def __init__(self, i, route, d):
        self.name = '[Train '+f"{i:2d}"+']'
        self.route = route
        self.blocks = d
        self.p = 0
        self.j = i
    
    def process(self):
        # Train departs depot to station
        yield env.timeout(60)
        
        here = self.route[0]   # Starting location
        
        # pass
        h = self.passengers(env, here)
        self.p += h
        
        print(f"{now():s} {self.name:s} {h:d} passenger(s) boarding at {here:s}")
        London_Birmingham.at[self.j, 'Train'] = self.name    # recording the train schedule in a dataframe
        London_Birmingham.at[self.j, 'Departure_time'] = env.now   # recording the train schedule in a dataframe
 
        i = 0
    
        for dest in self.route[1:]:
            data=df[df['From']==here]
            traveling_time=data.iloc[0].at['Traveling time']
            dwelltime=data.iloc[0].at['Dwell time']
            sig_block = data.iloc[0].at['Blocks']
            traveling_time_per_block = traveling_time / sig_block
            
            # Train requesting for signaling_block
            blocks = self.blocks[i]
            block_counter = 1
            
            print(f"{now():s} {self.name:s} {here:s} requesting for signaling block departing for {dest:s}")
            
            
            # Train travels to next station
            for block in blocks:
                with block.request() as req:
                    yield req
                    
                    print(f"{now():s} {self.name:s} enters block {block_counter} for {dest:s}")
                    yield env.timeout(traveling_time_per_block)
                    
                    # Train arrives at the next station
                    if block_counter == len(blocks):
                        print(f"{now():s} {self.name:s} arr {dest:s}")
                    
                        h = self.passengers(env, here)
                        self.p += h
                        here=dest
                        print(f"{now():s} {self.name:s} {h:d} passengers boarding")
                        here=dest
                        
                        
                        if here == self.route[-1]:
                            print(f"{now():s} {self.name:s} has reached the final destination")
                            London_Birmingham.at[self.j, 'Arrival_time'] = env.now   # recording the train schedule in a dataframe
                            London_Birmingham.at[self.j, 'Passengers'] = self.p      # recording the train schedule in a dataframe 
                    
                    block_counter +=1
                    
            i+=1
            
            
    def passengers(self, env, here):
        p = 0
        s = env.now - 21600
        t = passenger_arrival[passenger_arrival['station']== here]
        for i in list(t['arr_times']):
              if i > 0  and i <= s:
                p += 1
                passenger_arrival.arr_times = passenger_arrival.arr_times.replace({i: 0})
        return p 

## Blocks
<a id='3.1'></a>

The block refers to the signaling block.

In [138]:
def block():
    blocks=[]
    stations=df['From'].to_list()
    stations+=[df['To'].to_list()[-1]]
    here = stations[0]# starting location
    
    for dest in stations[1:]:
        data=df[df['From']==here]
        signaling_block = data.iloc[0].at['Blocks']
        destb = []
        
        for i in range(signaling_block):
            block = simpy.Resource(env, capacity = 1)
            destb.append(block)
        blocks.append(destb)
        here=dest
    return blocks

## Utilities
<a id='3.2'></a>

The line function is where we instantiante the train class and call the process method.

In [139]:
def line(d, n, start=6*3600, stop=7*3600):
    timing = 3600/n
    stations=df['From'].to_list()
    stations+=[df['To'].to_list()[-1]]
    yield env.timeout(start-env.now) # the line starts operating at 6am
    for i in range(int((stop-start)/timing)):
        t=Train(i, stations, d)
        env.process(t.process())
        yield env.timeout(timing)

In [140]:
def daytime(t):
    t=int(t)
    return f"{t//3600:02d}:{(t%3600)//60:02d}:{t%60:02d}"
def now():
    return daytime(env.now)

# Simulation
<a id='4'></a>

The 'run_simulation' function runs a simulation of the system. It takes the number of trains and the number of blocks for the second stretch, i.e, London Old Oak Common to Birmingham Interchange.

In [141]:
def run_simulation(n, k):
    global env
    global London_Birmingham
    df.iloc[1,4] = k
    env = simpy.Environment()
    d = block()
    passenger_generator()
    env.process(line(d, n))
    env.run()
    
    return London_Birmingham

In [142]:
z = run_simulation(20, 10)

06:01:00 [Train  0] 8 passenger(s) boarding at London Euston
06:01:00 [Train  0] London Euston requesting for signaling block departing for London Old Oak Common
06:01:00 [Train  0] enters block 1 for London Old Oak Common
06:04:00 [Train  1] 15 passenger(s) boarding at London Euston
06:04:00 [Train  1] London Euston requesting for signaling block departing for London Old Oak Common
06:06:00 [Train  0] arr London Old Oak Common
06:06:00 [Train  0] 5 passengers boarding
06:06:00 [Train  0] London Old Oak Common requesting for signaling block departing for Birmingham Interchange
06:06:00 [Train  0] enters block 1 for Birmingham Interchange
06:06:00 [Train  1] enters block 1 for London Old Oak Common
06:07:00 [Train  2] 2 passenger(s) boarding at London Euston
06:07:00 [Train  2] London Euston requesting for signaling block departing for London Old Oak Common
06:09:06 [Train  0] enters block 2 for Birmingham Interchange
06:10:00 [Train  3] 23 passenger(s) boarding at London Euston
06:10:0

# Statistics
<a id='5'></a>

We pass in the train schedule from the simulation into the statistics function to get our result.

In [143]:
# function to return the average travel time 
def statistics(London_Birmingham):
    tot_avg_time = np.mean(London_Birmingham['Arrival_time'] - London_Birmingham['Departure_time'])
    return tot_avg_time

#function to return the total passengers
def p_statistics(London_Birmingham):
    passengers = np.sum(London_Birmingham['Passengers'])
    return passengers

# Optimization
<a id='6'></a>

## Monte-Carlo optimisation to minimise the overall average travelling time.
<a id='6.1'></a>

Parameters for the optimisation function.
n is the number of iterations,
xmin is the minimum number of trains,
xmax is the maximum number of trains,
ymin is the minimum number of trains,
ymax is the maximum number of trains.

The solution outputs 4 numbers. Based on the number of trains and the number of signaling blocks it returns the minimum average travel time.

In [144]:
def minimise_avg_travl_time(n, xmin, xmax, ymin, ymax):
    x = [ random.randint(xmin, xmax) for i in range(n)]
    y = [ random.randint(ymin, ymax) for i in range(n)]
    xp = [ x[0] ]
    yp = [ y[0] ]
    simulation = run_simulation(xp[0], yp[0])
    fmin = statistics(simulation)
    
    for i in range(1, len(x)):
        sim = run_simulation(x[i], y[i]) 
        fi = statistics(sim)
        if fi < fmin:
            xp += [x[i]]
            yp += [y[i]]
            fmin = fi

    simulation = run_simulation(xp[-1], yp[-1])
    f = statistics(simulation)

    return len(xp), xp[-1], yp[-1], f

In [145]:
solution_1 = minimise_avg_travl_time(100,1,20,1,15)

06:01:00 [Train  0] 6 passenger(s) boarding at London Euston
06:01:00 [Train  0] London Euston requesting for signaling block departing for London Old Oak Common
06:01:00 [Train  0] enters block 1 for London Old Oak Common
06:06:00 [Train  0] arr London Old Oak Common
06:06:00 [Train  0] 31 passengers boarding
06:06:00 [Train  0] London Old Oak Common requesting for signaling block departing for Birmingham Interchange
06:06:00 [Train  0] enters block 1 for Birmingham Interchange
06:06:27 [Train  1] 3 passenger(s) boarding at London Euston
06:06:27 [Train  1] London Euston requesting for signaling block departing for London Old Oak Common
06:06:27 [Train  1] enters block 1 for London Old Oak Common
06:08:35 [Train  0] enters block 2 for Birmingham Interchange
06:11:10 [Train  0] enters block 3 for Birmingham Interchange
06:11:27 [Train  1] arr London Old Oak Common
06:11:27 [Train  1] 28 passengers boarding
06:11:27 [Train  1] London Old Oak Common requesting for signaling block departi

In [146]:
solution_1

(6, 12, 10, 3444.0)

## Monte-Carlo optimisation to maximize passengers throughput.
<a id='6.2'></a>

Similarly, the solution outputs 4 numbers. Based on the number of trains and the number of signaling blocks, it returns the maximum number of passengers.

In [147]:
def maximise_passengers(n, xmin, xmax, ymin, ymax):
    x = [ random.randint(xmin, xmax) for i in range(n)]
    y = [ random.randint(ymin, ymax) for i in range(n)]
    xp = [ x[0] ]
    yp = [ y[0] ]
    simulation = run_simulation(xp[0], yp[0])
    fmin = p_statistics(simulation)
    
    for i in range(1, len(x)):
        sim = run_simulation(x[i], y[i]) 
        fi = p_statistics(sim)
        if fi > fmin:
            xp += [x[i]]
            yp += [y[i]]
            fmin = fi

    simulation = run_simulation(xp[-1], yp[-1])
    f = p_statistics(simulation)

    return len(xp), xp[-1], yp[-1], f

In [148]:
solution_2 = maximise_passengers(100,1,20,1,15)

06:01:00 [Train  0] 4 passenger(s) boarding at London Euston
06:01:00 [Train  0] London Euston requesting for signaling block departing for London Old Oak Common
06:01:00 [Train  0] enters block 1 for London Old Oak Common
06:06:00 [Train  0] arr London Old Oak Common
06:06:00 [Train  0] 26 passengers boarding
06:06:00 [Train  0] London Old Oak Common requesting for signaling block departing for Birmingham Interchange
06:06:00 [Train  1] 0 passenger(s) boarding at London Euston
06:06:00 [Train  1] London Euston requesting for signaling block departing for London Old Oak Common
06:06:00 [Train  0] enters block 1 for Birmingham Interchange
06:06:00 [Train  1] enters block 1 for London Old Oak Common
06:08:12 [Train  0] enters block 2 for Birmingham Interchange
06:10:25 [Train  0] enters block 3 for Birmingham Interchange
06:11:00 [Train  1] arr London Old Oak Common
06:11:00 [Train  1] 25 passengers boarding
06:11:00 [Train  1] London Old Oak Common requesting for signaling block departi

In [149]:
solution_2

(3, 10, 1, 3057)

# References
<a id='7'></a>

[1] Simpy, "SimPy in 10 minutes," 2020. [Online]. Available: [https://simpy.readthedocs.io/]("https://simpy.readthedocs.io/en/latest/contents.html") [Accessed on: Apr. 18, 2020].

[2] A. Fajemisin, (2020). Introduction to Simulation: Possion [Source code]. Available: [https://moodle.ncirl.ie/mod/resource/view.php?id=55044.]("https://moodle.ncirl.ie/mod/resource/view.php?id=55044")
 
[3] A. Fajemisin, (2020). London Underground [Source code]. Available: [https://moodle.ncirl.ie/pluginfile.php/576329/mod_resource/content/0/London%20Underground%20v2.ipynb](https://moodle.ncirl.ie/pluginfile.php/576329/mod_resource/content/0/London%20Underground%20v2.ipynb).

[4] A. Fajemisin, (2020). MetaHeuristics [PDF]. Available: [https://moodle.ncirl.ie/mod/resource/view.php?id=51486](https://moodle.ncirl.ie/mod/resource/view.php?id=51486).